In [85]:
import numpy as np
import altair as alt 
import pandas as pd

In [86]:
def init_parameters(n_features, n_neurons, n_output):
    """
    Inicializa los parámetros del modelo (pesos y sesgos) para una red neuronal
    de una capa oculta y una capa de salida.

    Args:
        n_features (int): Número de características de entrada.
        n_output (int): Número de neuronas en la capa de salida.

    Returns:
        dict: Diccionario con los pesos y sesgos (W1, b1, W2, b2).
    """

    np.random.seed(100)
    W1 = np.random.uniform(size = (n_features, n_neurons))
    b1 = np.random.uniform(size = (1, n_neurons))

    W2 = np.random.uniform(size = (n_neurons, n_output))
    b2 = np.random.uniform(size = (1, n_output))

    return {
        "W1" : W1 
        , "b1" : b1 
        , "W2" : W2
        , "b2" : b2 
    }

In [87]:
def linear_function(W, X, b):
    """
    Calcula la combinación lineal entre pesos, entradas y sesgo: Z = XW + b.

    Args:
        W (np.array): Matriz de pesos.
        X (np.array): Datos de entrada.
        b (np.array): Vector de sesgos.

    Returns:
        np.array: Resultado de la combinación lineal.
    """
    return (X @ W)+ b 

In [88]:
def sigmoid_func(Z):
    """
    Aplica la función sigmoide elemento a elemento.

    Args:
        Z (np.array): Valores de entrada (logits).

    Returns:
        np.array: Activaciones entre 0 y 1.
    """ 
    return 1 / (1 + np.exp(-Z))

In [89]:
def cost_function(A, y):
    """
    Calcula la función de costo Binary Cross-Entropy para clasificación binaria.

    Args:
        A (np.array): Predicciones del modelo (entre 0 y 1).
        y (np.array): Etiquetas verdaderas (0 o 1).

    Returns:
        float: Valor promedio de la pérdida.
    """
    epsilon = 1e-15
    A = np.clip(A, epsilon, 1 - epsilon)
    return -np.mean(y * np.log(A) + (1 - y) * np.log(1 - A))

In [90]:
def predict(X, W1, W2, b1, b2):
    """
    Realiza una predicción binaria usando el modelo entrenado.

    Args:
        X (np.array): Conjunto de datos de entrada.
        W1, W2 (np.array): Pesos de las capas oculta y de salida.
        b1, b2 (np.array): Sesgos de las capas oculta y de salida.

    Returns:
        np.array: Predicciones binarias (0 o 1).
    """
    Z1 = linear_function(W1, X, b1)
    S1 = sigmoid_func(Z1)
    Z2 = linear_function(W2, S1, b2)
    S2 = sigmoid_func(Z2)
    return np.where(S2 >= 0.5, 1, 0)

In [91]:
def fit(X, y, n_features = 2, n_neurons = 3, n_output = 1, iterations = 10, eta = 0.001):
    """
    Entrena el modelo mediante el algoritmo de retropropagación y descenso por gradiente.

    Args:
        X (np.array): Matriz de características de entrada.
        y (np.array): Etiquetas reales.
        n_features (int): Número de entradas por ejemplo.
        n_neurons (int): Neuronas en la capa oculta.
        n_output (int): Neuronas en la capa de salida.
        iterations (int): Número de épocas de entrenamiento.
        eta (float): Tasa de aprendizaje.

    Returns:
        errors (list): Lista de errores durante el entrenamiento.
        params (dict): Diccionario con los pesos y sesgos finales.
    """

    params = init_parameters(
        n_features= n_features
        , n_neurons= n_neurons
        , n_output= n_output
    )

    errors = [] 

    for _ in range(iterations): 

        Z1 = linear_function(params['W1'], X, params['b1'])
        S1 = sigmoid_func(Z1)
        Z2 = linear_function(params['W2'], S1, params['b2'])
        S2 = sigmoid_func(Z2)

        error = cost_function(S2, y)
        errors.append(error)

        delta2 = S2 - y

        W2_gradients = S1.T @ delta2 
        params["W2"] = params["W2"] - W2_gradients * eta

        params["b2"] = params["b2"] - np.sum(delta2, axis = 0, keepdims= True) * eta 

        delta1 = (delta2 @ params["W2"].T) * S1 * (1 - S1)
        W1_gradients = X.T @ delta1 
        params["W1"] = params["W1"] - W1_gradients * eta 

        params["b1"] = params["b1"] - np.sum(delta1, axis = 0, keepdims= True) * eta 

    return errors, params 


        

In [92]:
y = np.array([[0, 1, 1, 0]]).T 
X = np.array([[0, 0, 1, 1]
              ,[0, 1, 0, 1]]).T 

In [93]:
errors, params = fit(X, y, iterations=3000, eta=0.1)

In [94]:
y_pred = predict(X, params["W1"], params["W2"], params["b1"], params["b2"])
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Multi-layer perceptron accuracy: %.2f%%' % accuracy)

Multi-layer perceptron accuracy: 100.00%


In [95]:
alt.data_transformers.disable_max_rows()
df = pd.DataFrame({"errors":errors, "time-step": np.arange(0, len(errors))})
alt.Chart(df).mark_line().encode(x="time-step", y="errors").properties(title='Chart 2')

alt.Chart(...)

# Mismo Dataset en Framework Pytorch

In [96]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import altair as alt

In [97]:
# Inputs: XOR problem
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T  # Shape (4, 2)
y = np.array([[0, 1, 1, 0]]).T  # Shape (4, 1)

# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Define the model
class SimpleNet(nn.Module):
    """
    Red neuronal simple con una capa oculta para resolver el problema XOR.

    Arquitectura:
    - Capa de entrada: 2 neuronas (una por cada feature)
    - Capa oculta: 16 neuronas con activación sigmoide
    - Capa de salida: 1 neurona con activación sigmoide (para clasificación binaria)
    """
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        Propagación hacia adelante del modelo.

        Args:
            x (torch.Tensor): Tensor de entrada de tamaño (batch_size, 2)

        Returns:
            torch.Tensor: Salida del modelo (probabilidad entre 0 y 1)
        """
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = SimpleNet()

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
errors_pytorch = []
epochs = 3000

for epoch in range(3000):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    errors_pytorch.append(loss.item())

# Convert errors to DataFrame and plot
df2 = pd.DataFrame({"errors": errors_pytorch, "time-step": np.arange(epochs)})

alt.Chart(df2).mark_line().encode(
    x="time-step",
    y="errors"
).properties(title='Chart 3')



alt.Chart(...)

In [98]:
"""
Comparación visual de la función de pérdida entre dos implementaciones: 
una red neuronal desde cero con NumPy y otra usando PyTorch.
"""


errors_numpy = errors

df_combined = pd.DataFrame({
    "time-step": np.tile(np.arange(epochs), 2),
    "error": errors_numpy + errors_pytorch,
    "modelo": ["NumPy"] * epochs + ["PyTorch"] * epochs
})

alt.Chart(df_combined).mark_line().encode(
    x="time-step",
    y="error",
    color="modelo"
).properties(
    title="Comparación de la función de pérdida durante el aprendizaje"
)

alt.Chart(...)

### Respuestas a las preguntas

1. **¿Existen cambios de arquitectura en las 2 redes implementadas?**  
   Sí, existen diferencias notables en la arquitectura entre ambas redes neuronales:  
   - La red en NumPy tiene una sola capa oculta con 3 neuronas y usa la activación sigmoide.
   - La red en PyTorch también tiene una sola capa oculta, pero con 16 neuronas, lo que le otorga mayor capacidad de representación.  
   Aunque ambas tienen la misma estructura general (entrada → oculta → salida), difieren en complejidad por el número de neuronas y el uso de optimizadores (descenso por gradiente simple en NumPy vs. Adam en PyTorch).

2. **¿Existen diferencias en la velocidad de convergencia entre las 2 redes?**  
   En la ejecución actual, la red de NumPy converge más rápidamente que la de PyTorch, como se observa en la gráfica comparativa. Sin embargo, este resultado no es determinista: la velocidad de convergencia puede cambiar entre ejecuciones debido a la inicialización aleatoria de los pesos, especialmente en PyTorch (donde no se ha fijado la semilla).  
   Por tanto, no se puede afirmar de forma general que una red converge siempre más rápido que la otra dependerá de factores como la arquitectura, la tasa de aprendizaje, el optimizador y la inicialización.

3. **En la función init_parameters, ¿qué sucede si inicializamos los pesos en 0 en lugar de valores aleatorios? ¿y los bias?**  
   - Inicializar los pesos en 0 impide que las neuronas aprendan de manera independiente, ya que todas calculan exactamente lo mismo y se actualizan igual en cada iteración. Esto genera un problema de simetría que bloquea el aprendizaje.
   - Inicializar los bias en 0 no causa ese problema, ya que los sesgos se ajustan de manera individual durante el entrenamiento. Por ello, es común inicializarlos en cero sin afectar la capacidad de la red.